In [ ]:
# Install required packages on Databricks cluster
%pip install openai pydantic python-dotenv

In [ ]:
# Restart Python kernel to load new packages
dbutils.library.restartPython()

In [ ]:
# Import required modules
from pydantic import BaseModel, Field, field_validator
from datetime import datetime
import json
from pathlib import Path

# ==========================================
# PYDANTIC SCHEMAS - Definice struktury dat
# ==========================================
# Tyto třídy definují OČEKÁVANÝ FORMÁT výstupu z LLM.
# Pydantic zajišťuje:
# 1. Type safety - každé pole má jasný datový typ (str, list[str], dict...)
# 2. Validation - automatická kontrola formátu (např. ISO 8601 datum)
# 3. Default values - pokud LLM pole vynechá, použije se výchozí hodnota
# 4. Field descriptions - metadata pro LLM prompt (vysvětluje, co pole obsahuje)

class ProjectMetadata(BaseModel):
    """Metadata o business projektu.

    Obsahuje základní identifikační údaje:
    - project_name: Název projektu (např. "Supplier Risk Insights 2.0")
    - sponsor: Jméno sponzora/zadavatele (např. "Marek Hrubý")
    - submitted_at: Datum podání žádosti ve formátu ISO 8601 (YYYY-MM-DD)
    - extra: Slovník pro dodatečná pole (např. "oddělení", "priorita")

    Pydantic automaticky validuje, že všechna pole mají správný typ.
    """

    project_name: str = Field(description="Name of the project")
    sponsor: str = Field(description="Name of the project sponsor")
    submitted_at: str = Field(description="Date when the request was submitted, in ISO 8601 format (YYYY-MM-DD)")
    extra: dict[str, str] = Field(default_factory=dict, description="Additional metadata fields as key-value pairs")

    @field_validator('submitted_at')
    @classmethod
    def validate_iso_date(cls, v: str) -> str:
        """Custom validator: Zkontroluje, že datum je ve správném ISO 8601 formátu.

        Jak to funguje:
        1. Pydantic zavolá tuto funkci PO kontrole typu (že 'v' je str)
        2. Pokusíme se datum parsovat pomocí datetime.fromisoformat()
        3. Pokud úspěch → vrátíme původní string (validace OK)
        4. Pokud chyba → vyhodíme ValueError s jasnou chybovou zprávou

        Příklad:
        - "2025-10-28" → ✅ projde
        - "28.10.2025" → ❌ ValueError
        - "unknown" → ❌ ValueError
        """
        try:
            datetime.fromisoformat(v)
            return v
        except ValueError:
            raise ValueError(f"Date must be in ISO 8601 format (YYYY-MM-DD), got: {v}")


class BusinessRequest(BaseModel):
    """Kompletní struktura parsovaného business dokumentu.

    Obsahuje 9 hlavních částí business požadavku:
    1. project_metadata - základní údaje o projektu (nested object)
    2. goal - hlavní cíl projektu (string)
    3. scope_in - co JE v rozsahu (string)
    4. scope_out - co NENÍ v rozsahu (string)
    5. entities - klíčové business entity (array of strings)
    6. metrics - KPI metriky (array of strings)
    7. sources - datové zdroje (array of strings)
    8. constraints - omezení a požadavky (array of strings)
    9. deliverables - požadované výstupy (array of strings)

    Default hodnoty:
    - Stringy: "unknown" (pokud LLM pole nenajde)
    - Listy: [] (prázdný array místo None)

    Díky Pydantic můžeme:
    - validated = BusinessRequest(**json_dict) → automatická validace
    - validated.model_dump() → zpět do dict pro uložení do JSON
    - validated.project_metadata.sponsor → type-safe přístup k polím
    """

    project_metadata: ProjectMetadata = Field(description="Project metadata including name, sponsor, and submission date")
    goal: str = Field(default="unknown", description="Main goal or objective of the project")
    scope_in: str = Field(default="unknown", description="What is included in the project scope")
    scope_out: str = Field(default="unknown", description="What is explicitly excluded from the project scope")
    entities: list[str] = Field(default_factory=list, description="Key business entities involved in the project")
    metrics: list[str] = Field(default_factory=list, description="Key metrics or KPIs to be tracked")
    sources: list[str] = Field(default_factory=list, description="Expected data sources for the project")
    constraints: list[str] = Field(default_factory=list, description="Constraints, limitations, or special requirements")
    deliverables: list[str] = Field(default_factory=list, description="Required deliverables or artifacts from the project")

print("✅ Schemas defined successfully")


## 1. Configure Azure OpenAI with Databricks Secrets

**Setup Instructions:**
```bash
# Create secret scope (run once in Databricks CLI or UI)
databricks secrets create-scope --scope mcop

# Add Azure OpenAI credentials
databricks secrets put --scope mcop --key azure-openai-endpoint
databricks secrets put --scope mcop --key azure-openai-api-key
databricks secrets put --scope mcop --key azure-openai-deployment-name
```

**Expected values:**
- `azure-openai-endpoint`: https://minar-mhi2wuzy-swedencentral.cognitiveservices.azure.com/openai/v1/
- `azure-openai-deployment-name`: test-gpt-5-mini

In [ ]:
# Get Azure configuration from Databricks secrets
AZURE_ENDPOINT = dbutils.secrets.get(scope="mcop", key="azure-openai-endpoint").strip()
AZURE_API_KEY = dbutils.secrets.get(scope="mcop", key="azure-openai-api-key").strip()
DEPLOYMENT_NAME = dbutils.secrets.get(scope="mcop", key="azure-openai-deployment-name").strip()

if not all([AZURE_ENDPOINT, AZURE_API_KEY, DEPLOYMENT_NAME]):
    raise ValueError("Missing Azure configuration in Databricks secrets (scope: mcop)")

print(f"☁️ Azure OpenAI configured from Databricks secrets")
print(f"   Endpoint: {AZURE_ENDPOINT}")
print(f"   Deployment: {DEPLOYMENT_NAME}")


## 2. Load Sample Business Document

In [ ]:
# Hardcoded sample business document
business_document = """# Žádost o datový projekt – Supplier Risk Insights 2.0

## Projekt
**Název:** Supplier Risk Insights 2.0
**Sponzor:** Marek Hrubý (VP Procurement Excellence)
**Datum:** 2025-10-28
**Oddělení:** Group Procurement Analytics
**Priorita:** Kritická – Q4 OKR "Stabilizace dodavatelského řetězce"

## Cíl
Dodat konsolidovaný pohled na spolehlivost dodavatelů napříč BA/BS datamarťy a SAP ECC zdroji. Výsledný reporting musí upozorňovat na dodavatele s rostoucím lead time, častými reklamacemi nebo blokacemi plateb, aby procurement dokázal včas přesměrovat objem a eskalovat smluvní pokuty.

## Rozsah

### In Scope
- Historická data o purchase orders (posledních 36 měsíců) včetně RU/DE regionu.
- Dimenze dodavatel, produkt, dodací lokace, nákupní organizace.
- SLA metriky: on-time delivery, defect rate, invoice dispute count.
- Spárování se security klasifikací (Confidential vs Internal).
- Export KPI do Power BI workspace "Supplier Control Tower".

### Out of Scope
- Forecasting budoucích objednávek (řeší Supply Planning tým).
- Integrace s CRM a risk ratingy třetích stran.
- Real-time streaming ze SCADA nebo IoT senzorů.
- Detailní finanční marže – používá Finance Controlling.

## Klíčové entity & metriky

### Entity
- Supplier Master (Collibra/Unity Catalog `dimv_supplier`).
- Purchase Order Header + Item (`factv_purchase_order`, `factv_purchase_order_item`).
- Quality Incident (`factv_quality_notification`).
- Delivery Calendar Dimension (`dimv_delivery_date`).

### Metriky
- Supplier Reliability Index (vážený mix on-time %, dispute rate, defect rate).
- Average Goods Receipt Lead Time (dny).
- % PO s „blocked for payment" statusem.
- NCR Count (non-conformance reports) za poslední kvartál.
- Spend concentration top 10 dodavatelů.

## Očekávané zdroje
- Databricks Unity Catalog: `dm_ba_purchase`, `dm_bs_purchase` schemata.
- Collibra Data Catalog export (zajišťuje lineage a vlastníky).
- SAP ECC tabulky: `EKKO`, `EKPO`, `LFA1`, `MKPF`.
- SharePoint složka "Supplier Audits" pro manuální NCR zápisy.

## Omezení
- GDPR: žádná osobní data supplier kontaktů v datasetu; pseudonymizace ID.
- Data retention: pouze 3 roky historie v produkčním modelu.
- Každý dashboard refresh < 5 min, jinak neprojde SLA.
- Row Level Security podle regionu (EMEA, AMER, APAC).
- Pouze read-only přístup do SAP; žádné zápisy zpět.

## Požadované artefakty
- Kurátorované `business_request.json` a `structure.json` pro Tool 3/7.
- Quality report shrnující articulationScore + missingFromSource flagy.
- Power BI semantic model + definice DAX measures.
- Governance runbook popisující validace a kontakty (owner, steward).
- Checklist P0/P1/P2 mitigací pro Supplier Risk komisi.
"""

print(f"📄 Business document loaded ({len(business_document)} characters)")
print("\nFirst 300 characters:")
print("=" * 60)
print(business_document[:300])
print("...")

## 3. Parse Document Using Azure OpenAI

In [ ]:
# ==========================================
# AZURE OPENAI CLIENT - Inicializace LLM
# ==========================================
from openai import AzureOpenAI

print(f"🔄 Parsing document with Azure OpenAI ({DEPLOYMENT_NAME})...")

# System prompt - instrukce pro LLM, jak má parsovat dokument
# Popisuje:
# - Jaký je úkol (extract structured information)
# - Jaké sekce očekáváme (Projekt, Cíl, Rozsah...)
# - Jak má LLM formátovat výstup (JSON, ISO 8601 datumy, arrays...)
# - Co dělat, když něco chybí (použít "unknown")
SYSTEM_PROMPT = """You are a business requirements parser. Your task is to extract structured information from business request documents.

Documents may contain a mix of Czech and English. Common section headers include:
- "Projekt" / "Project" - project metadata (name, sponsor, date)
- "Cíl" / "Goal" - main project objective
- "Rozsah" / "Scope" - what is in/out of scope
- "Klíčové entity & metriky" / "Key entities & metrics" - business entities and KPIs
- "Očekávané zdroje" / "Expected sources" - data sources
- "Omezení" / "Constraints" - limitations and requirements
- "Požadované artefakty" / "Required artifacts" - deliverables

IMPORTANT INSTRUCTIONS:
1. Extract information into the structured JSON format exactly as specified
2. Use "unknown" for any missing sections
3. Ensure dates are in ISO 8601 format (YYYY-MM-DD)
4. Extract lists as arrays of strings, not concatenated text
5. For project metadata, look for project name, sponsor name, and submission date
6. Any additional metadata fields should go into the "extra" dictionary
7. Be thorough - extract all relevant information from the document
8. Return ONLY valid JSON, no markdown or code blocks

Expected JSON schema:
{
  "project_metadata": {
    "project_name": "string",
    "sponsor": "string",
    "submitted_at": "YYYY-MM-DD",
    "extra": {}
  },
  "goal": "string",
  "scope_in": "string",
  "scope_out": "string",
  "entities": [],
  "metrics": [],
  "sources": [],
  "constraints": [],
  "deliverables": []
}
"""

# ==========================================
# AZURE OPENAI CLIENT SETUP
# ==========================================
# Důležité: AzureOpenAI SDK vyžaduje endpoint BEZ /openai/v1/ suffixu
# SDK automaticky přidá správnou cestu podle API version
azure_endpoint = AZURE_ENDPOINT.replace("/openai/v1/", "").replace("/openai/v1", "").rstrip("/")

client = AzureOpenAI(
    azure_endpoint=azure_endpoint,      # https://minar-mhi2wuzy-swedencentral.cognitiveservices.azure.com/
    api_key=AZURE_API_KEY,               # API klíč z Databricks secrets
    api_version="2024-10-21"             # Azure OpenAI API verze
)

# User message - spojíme business dokument s instrukcemi
user_message = f"""Parse the following business request document:

{business_document}

Extract all information into the structured JSON format."""

# ==========================================
# LLM CALL - Volání Azure OpenAI
# ==========================================
# Klíčové parametry:
# - model: deployment name (test-gpt-5-mini)
# - messages: system prompt + user message (standard Chat Completion format)
# - response_format: {"type": "json_object"} → LLM MUSÍ vrátit valid JSON
#
# JSON mode zajišťuje:
# 1. LLM vždycky vrátí parsovatelný JSON (ne markdown, ne prose text)
# 2. Nutné explicitně říct v system promptu "return JSON"
# 3. Pydantic pak JSON validuje proti našemu schématu
response = client.chat.completions.create(
    model=DEPLOYMENT_NAME,
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_message}
    ],
    response_format={"type": "json_object"}  # ✅ KRITICKÉ: Vynutí JSON output
)

# Extract JSON string from response
raw_response = response.choices[0].message.content

# ==========================================
# PYDANTIC VALIDATION - Kontrola struktury
# ==========================================
try:
    # 1. Parse JSON string → Python dict
    parsed_json = json.loads(raw_response)

    # 2. Validate dict against Pydantic schema
    #    Toto provede:
    #    - Type checking (str? list? dict?)
    #    - Custom validators (@field_validator pro datum)
    #    - Default value injection (pokud pole chybí)
    #    - Vyhodí ValidationError, pokud struktura nesedí
    validated = BusinessRequest(**parsed_json)

    # 3. Convert validated model back to dict (pro uložení)
    #    model_dump() vrátí čistý dict, ale už validovaný
    parsed_json = validated.model_dump()

    print("✅ Parsing complete!")
    print(f"   Model: {response.model}")
    print(f"   Tokens: {response.usage.total_tokens}")
    print(f"   Validation: ✅ Passed")

except json.JSONDecodeError as e:
    # JSON mode selhal → LLM vrátil nevalidní JSON (shouldn't happen)
    print(f"❌ JSON parsing error: {e}")
    print(f"Raw response: {raw_response}")
    raise

except Exception as e:
    # Pydantic validation selhala → JSON má špatnou strukturu
    # (např. chybí povinné pole, špatný typ, custom validator error)
    print(f"❌ Validation error: {e}")
    print(f"Parsed JSON: {parsed_json}")
    raise

# Full prompt for audit (save to DBFS later)
prompt_used = f"System: {SYSTEM_PROMPT}\n\nUser: {user_message}"


## 4. Display Parsed JSON

In [ ]:
# Display parsed JSON
print("📊 Parsed Business Request:")
print("=" * 60)
print(json.dumps(parsed_json, indent=2, ensure_ascii=False))

# Also show as Pydantic model
print("\n" + "=" * 60)
print("📋 Validation:")
try:
    validated = BusinessRequest.model_validate(parsed_json)
    print(f"✅ Schema valid: {validated.project_metadata.project_name}")
    print(f"   Sponsor: {validated.project_metadata.sponsor}")
    print(f"   Date: {validated.project_metadata.submitted_at}")
    print(f"   Entities: {len(validated.entities)} found")
    print(f"   Sources: {len(validated.sources)} found")
except Exception as e:
    print(f"❌ Validation error: {e}")

## 5. Save Results to DBFS

Save to `/dbfs/FileStore/mcop/tool0_samples/` for persistence.

In [ ]:
# Save results to DBFS
timestamp = datetime.now().isoformat().replace(':', '-')  # DBFS-safe filename
output_dir = Path('/dbfs/FileStore/mcop/tool0_samples')
output_dir.mkdir(parents=True, exist_ok=True)

# Save JSON result
json_path = output_dir / f"{timestamp}.json"
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(parsed_json, f, indent=2, ensure_ascii=False)

# Save prompt
md_path = output_dir / f"{timestamp}.md"
with open(md_path, 'w', encoding='utf-8') as f:
    f.write(f"# Parse Request - {timestamp}\n\n")
    f.write(f"## Prompt Used\n\n```\n{prompt_used}\n```\n\n")
    f.write(f"## Raw Response\n\n```\n{raw_response}\n```\n\n")
    f.write(f"## Parsed JSON\n\n```json\n{json.dumps(parsed_json, indent=2, ensure_ascii=False)}\n```\n")

print(f"💾 Results saved to DBFS:")
print(f"   JSON: {json_path}")
print(f"   Markdown: {md_path}")
print(f"\n📂 View files in Databricks:")
print(f"   dbfs:/FileStore/mcop/tool0_samples/")

## 6. Summary

✅ **Databricks Deployment Complete:**
- [x] Azure OpenAI credentials from Databricks secrets (scope: `mcop`)
- [x] Pydantic schemas for validation (ProjectMetadata, BusinessRequest)
- [x] Sample business document (Czech/English mix)
- [x] OpenAI SDK with Azure endpoint and JSON mode
- [x] Pydantic validation after parsing
- [x] Results saved to DBFS (`/dbfs/FileStore/mcop/tool0_samples/`)

**Databricks-Specific Changes:**
- ✅ `dbutils.secrets.get()` instead of `.env` file
- ✅ `/dbfs/FileStore/` paths instead of local `data/` directory
- ✅ `%pip install` cell for package installation
- ✅ `dbutils.library.restartPython()` to reload packages
- ✅ DBFS-safe filenames (replaced `:` with `-` in timestamps)

**Azure AI Foundry Configuration:**
- **Endpoint:** https://minar-mhi2wuzy-swedencentral.cognitiveservices.azure.com/openai/v1/
- **Deployment:** test-gpt-5-mini
- **Model:** gpt-5-mini-2025-08-07
- **Pattern A:** Direct OpenAI SDK with JSON mode + Pydantic validation

**Next Steps:**
1. Create secret scope: `databricks secrets create-scope --scope mcop`
2. Add credentials: `databricks secrets put --scope mcop --key azure-openai-endpoint`
3. Run notebook on Databricks cluster
4. Verify files in DBFS: `dbfs:/FileStore/mcop/tool0_samples/`